<a href="https://colab.research.google.com/github/awaris123/OSNA-Project2/blob/main/FakeNewsNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np
np.random.seed(0)

In [23]:
df=pd.read_csv("https://raw.githubusercontent.com/awaris123/OSNA-Project2/main/option1-data/train.csv")
df['label'].value_counts()

unrelated    175598
agreed        74238
disagreed      6606
Name: label, dtype: int64

In [24]:
'''
categorical_replacement = {'unrelated':0, 'agreed': 1, 'disagreed': 2}
df['label'].replace(categorical_replacement, inplace=True)
'''
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df['label'].values)



In [27]:
#df['label'].value_counts()
df['tmp']=integer_encoded

df['tmp'].value_counts()


2    175598
0     74238
1      6606
Name: tmp, dtype: int64

In [64]:
#visualize in graph format
label_num = {'agreed': 1, 'disagreed': 0}
edge_list= [(row[1], row[2], label_num[row[-1]])for row in df.values if row[-1]!='unrelated']

[nltk_data] Downloading package punkt to /Users/arushirai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arushirai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [65]:
import matplotlib.pyplot as plt

def visualize_edges(edge_list):
  G = nx.DiGraph()
  G.add_weighted_edges_from(edge_list)  
  plt.figure(figsize=(50,50))
  nx.draw(G, with_labels=True, font_weight='bold')
  plt.show()

In [66]:
# view distribution after removing stop words
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

sentence = """At eight o'clock on Thursday morning ... Arthur didn't feel very good."""

def filter_tokens(sentence):
  tokenizer = RegexpTokenizer(r'\w+')
  token_list = tokenizer.tokenize(sentence)
  filtered_tokens=[]
  for token in token_list:
    token = token.lower()
    if token not in stopwords.words('english'):
      filtered_tokens.append(token)
  return filtered_tokens

filter_tokens(sentence)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['eight', 'clock', 'thursday', 'morning', 'arthur', 'feel', 'good']

In [67]:
def unique_article_list(df):
  article_id=set()
  unique_articles=[]
  for index, row in df.iterrows():
    if row['tid2'] not in article_id:
      unique_articles.append(row['title2_en'])
      article_id.add(row['tid2'])
    if row['tid1'] not in article_id:
      unique_articles.append(row['title1_en'])
      article_id.add(row['tid1'])
  return unique_articles

In [68]:
unique_articles = unique_article_list(df)

In [69]:
corpus=[]
for article_text in unique_articles:
  corpus.extend(filter_tokens(article_text))


In [71]:
from collections import Counter 
Counter(corpus).most_common(10)

[('eat', 8704),
 ('rumors', 8327),
 ('old', 7962),
 ('year', 7782),
 ('rumor', 7683),
 ('new', 7442),
 ('people', 7114),
 ('one', 7011),
 ('years', 6951),
 ('weight', 6830)]

In [ ]:
# separate validation, holdout (0.2) and train set
import train_test_split from sklearn.model_selection
train_test_split( test_size=0.2)

In [13]:
from generate_embeddings import Preprocessing
Preprocessing("./GoogleNews-vectors-negative300.bin.gz")

TypeError: load() takes 2 positional arguments but 3 were given

In [31]:
import pickle
with open("train_df_w_embeddings.pickle", 'rb') as f:
    df = pickle.load(f)

In [32]:
df['other']=df['joint_embedding'].apply(lambda x: x.numpy()[0])

df.head()


,id,tid1,tid2,title1_en,title2_en,label,joint_embedding,other
0,195611,0,1,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated,"((tf.Tensor(-0.005404385653409091, shape=(), d...","[-0.005404385653409091, -0.02629852294921875, ..."
1,191474,2,3,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated,"((tf.Tensor(0.022618974958147322, shape=(), dt...","[0.022618974958147322, 0.034480503627232144, -..."
2,25300,2,4,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated,"((tf.Tensor(0.022618974958147322, shape=(), dt...","[0.022618974958147322, 0.034480503627232144, -..."
3,123757,2,8,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP overtakes Hong Kong? Bureau of ...,unrelated,"((tf.Tensor(0.022618974958147322, shape=(), dt...","[0.022618974958147322, 0.034480503627232144, -..."
4,141761,2,11,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outpaces Hong Kong? Defending R...,unrelated,"((tf.Tensor(0.022618974958147322, shape=(), dt...","[0.022618974958147322, 0.034480503627232144, -..."


In [41]:
def filter_nans(arr):
    #arr=np.asarray(arr)#.astype(np.float64)
    return np.isnan(sum(arr))

# load data
with open("train_df_w_embeddings.pickle", 'rb') as f:
    df = pickle.load(f)
    df['joint_embedding']=df['joint_embedding'].apply(lambda x: x.numpy()[0].tolist())
    df['filter']=df['joint_embedding'].apply(lambda x: filter_nans(x))


In [42]:
df[df['filter']==True]

,id,tid1,tid2,title1_en,title2_en,label,joint_embedding,filter
738,228516,480,101503,"i am eating, do n't love sports 70 days of fre...",= = = = = = = = = = = = = = = = = = = = = = = ...,unrelated,"[-0.0014245169503348215, 0.028203691755022322,...",True
2205,91800,1418,18118,"starting from july 1, shenzhen 's number three...","""ZIBU"": ""ZIBU"" is 20.",unrelated,"[0.001885986328125, 0.03927001953125, -0.01633...",True
4033,176365,2523,4846,New regulation 2018: Drinking without driving ...,"No, no, no, no, no, no, no, no.",unrelated,"[-0.011182512555803572, 0.009141104561941964, ...",True
6158,157136,3884,7220,The new change of land contract policy in 2018...,\n\n \n,unrelated,"[-0.017295143821022728, -0.013334794477982954,...",True
7409,129338,4842,4846,"New regulation, do not want to lose points qui...","No, no, no, no, no, no, no, no.",unrelated,"[-0.011182512555803572, 0.009141104561941964, ...",True
...,...,...,...,...,...,...,...,...
235207,12902,154277,51650,Why is it more and more grey?,More gray hairs.,agreed,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",True
240007,112406,157348,157430,Qin Jongjie and Yuan Binyan are very close to ...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,unrelated,"[0.10481770833333333, 0.009668986002604166, 0....",True
246093,145161,160918,133639,Tencent Tencent King Glory team annual bonus 1...,Alipay rumour. He didn't do it. He didn't do it.,unrelated,"[0.0021209716796875, 0.010037740071614584, 0.0...",True
246424,198267,161009,33924,rumor is that chengguan bridge in chongqing is...,It's a rumour.,unrelated,"[-0.001324462890625, 0.1552490234375, -0.18129...",True
